In [1]:
from pinecone import Pinecone

pc = Pinecone(api_key="09783e77-bcef-4448-a649-966eb17480d4")
index = pc.Index("mulitlangone")

c:\Users\mazen\Desktop\araEmbed\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [38]:
import json
from pinecone import Pinecone as pc

# Load your data from dataChatbot2.json
with open('dataChatbot2.json', 'r', encoding='utf-8') as f:
    surah_data = json.load(f)
    surah_data = surah_data[:20]  
# Initialize Pinecone
pc = pc(api_key="09783e77-bcef-4448-a649-966eb17480d4")
index = pc.Index("mulitlangone")

# Prepare the data into smaller chunks
data = []
for surah in surah_data:
    surah_id = surah["id"]
    surah_name = surah["Surah"]
    
    # Create chunks (e.g., Ayat, Naming Reason, Virtue, Meanings, etc.)
    chunks = [
        {"id": f"{surah_id}_ayat", "text": surah["Ayat"]},  # آيات
        {"id": f"{surah_id}_naming", "text": surah["Naming Reason"]},  # سبب التسمية
        {"id": f"{surah_id}_virtue", "text": " ".join(surah["Virtue"]), "metadata": {'نوع_المحتوى': 'الفضائل'}},  # الفضائل
        {"id": f"{surah_id}_meanings", "text": " ".join(surah["Meanings"])}  # المعاني
    ]
    
    # Add to data list
    data.extend(chunks)

# Generate embeddings for each chunk of text
embeddings = pc.inference.embed(
   model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)

# Create the list of vectors with metadata for upsertion into Pinecone
vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {
            'النص': d['text'],  # النص (Text)
            'نوع_المحتوى': d['id'].split('_')[1]  # نوع المحتوى (Content Type): آيات, سبب التسمية, الفضائل, المعاني
        }
    })
# Upsert vectors to Pinecone
index.upsert(
    vectors=vectors,
    namespace="ns1"
)


{'upserted_count': 80}

In [37]:
for d,e in zip(data,embeddings):
    print(d['id'].split('_')[1])

ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings
ayat
naming
virtue
meanings


In [46]:
query = "طب ايه افضال سوره الفاتحه" 
embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "passage"
    }
)

In [49]:
results = index.query(
    namespace="ns1",
    vector=embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': '7_naming',
              'metadata': {'النص': 'انْفِرَادُ السُّورَةِ بِذِكْرِ أَصْحَابِ '
                                   'الْأَعْرَافِ',
                           'نوع_المحتوى': 'naming'},
              'score': 0.84539485,
              'values': []},
             {'id': '20_naming',
              'metadata': {'النص': 'انْفِرَادُ السُّورَةِ بِمُفْتَتَحِ حَرْفَي '
                                   '(طَه) دُونَ غَيرِهَا مِنْ سُوَرِ '
                                   'القُرْآنِ؛ فَسُمِّيَتْ بِهِمَا',
                           'نوع_المحتوى': 'naming'},
              'score': 0.842599452,
              'values': []},
             {'id': '1_naming',
              'metadata': {'النص': 'لأنَّهُ تُفتَتَحُ بِكِتَابَتِهَا '
                                   'المَصِاحِفُ، وبقِرَاءَتِها الصَّلَوَاتُ',
                           'نوع_المحتوى': 'naming'},
              'score': 0.838806808,
              'values': []}],
 'namespace': 'ns1',
 'usage': {'read_units': 6}}